# Netflix recommendations

<div class="alert alert-block alert-warning">
Replace <code>YOUR_GITHUB_TOKEN</code> in the install script. To get your token follow the instructions in the <a href="../README.md">README.md</a>
</div>

## Boilerplate

In [ ]:
%pip install gcsfs pandas
%pip install  'https://us-central1-data-359211.cloudfunctions.net/github-proxy/superlinked-0.2.2-py3-none-any.whl?token=YOUR_GITHUB_TOKEN'

## Imports and constants

In [1]:
import pandas as pd

from datetime import timedelta

from superlinked.framework.common.dag.recency_node import PeriodTimeParam
from superlinked.framework.common.schema.schema import schema, String, Timestamp, IdField
from superlinked.framework.common.parser.dataframe_parser import DataFrameParser
from superlinked.framework.dsl.executor.in_memory.in_memory_executor import InMemoryExecutor, InMemoryApp
from superlinked.framework.dsl.index.index import Index
from superlinked.framework.dsl.query.param import Param
from superlinked.framework.dsl.query.query import Query
from superlinked.framework.dsl.query.result import Result
from superlinked.framework.dsl.source.in_memory_source import InMemorySource
from superlinked.framework.dsl.space.text_similarity_space import TextSimilaritySpace
from superlinked.framework.dsl.space.recency_space import RecencySpace

In [2]:
MODEL = "sentence-transformers/paraphrase-MiniLM-L3-v2"
YEAR_IN_DAYS = 365
TOP_N = 10
DATASET_URL = "https://storage.googleapis.com/superlinked-notebook-netflix-shows-dataset/titles.csv"

## Explore dataset

In [3]:
movie_df = pd.read_csv(DATASET_URL)
movie_df = movie_df[["description", "genres", "title", "release_year"]].drop_duplicates(subset=["description"]).dropna(how='any')
movie_df['id'] = movie_df["description"].map(hash)
movie_df["genres"] = movie_df["genres"].apply(lambda x: " ".join(eval(x)))
movie_df["timestamp"] = [pd.Timestamp(year=year, month=1, day=1).timestamp() for year in movie_df["release_year"].tolist()]
movie_df.head()

,description,genres,title,release_year,id,timestamp
0,This collection includes 12 World War II-era p...,documentation,Five Came Back: The Reference Films,1945,2176947496386237966,-788918400.0
1,A mentally unstable Vietnam War veteran works ...,drama crime,Taxi Driver,1976,-7356384711955127664,189302400.0
2,Intent on seeing the Cahulawassee River before...,drama action thriller european,Deliverance,1972,-6152454522568329380,63072000.0
3,"King Arthur, accompanied by his squire, recrui...",fantasy action comedy,Monty Python and the Holy Grail,1975,-4106506009191763053,157766400.0
4,12 American military prisoners in World War II...,war action,The Dirty Dozen,1967,2766439547854098643,-94694400.0


## Set up Superlinked

In [4]:
@schema
class MovieSchema:
    description: String
    title: String
    release_timestamp: Timestamp
    genres: String
    id: IdField

In [5]:
movie = MovieSchema()

In [6]:
description_space = TextSimilaritySpace(text=movie.description, model=MODEL)
title_space = TextSimilaritySpace(text=movie.title, model=MODEL)
genre_space = TextSimilaritySpace(text=movie.genres, model=MODEL)
recency_space = RecencySpace(timestamp=movie.release_timestamp, period_time_param_list=[
    PeriodTimeParam(timedelta(days=4 * YEAR_IN_DAYS), negative_filter=0), 
    PeriodTimeParam(timedelta(days=10 * YEAR_IN_DAYS), negative_filter=0), 
    PeriodTimeParam(timedelta(days=40 * YEAR_IN_DAYS), negative_filter=0)
])

In [7]:
movie_index = Index(spaces=[description_space, title_space, genre_space, recency_space])

In [8]:
query_text_param = Param("query_text")

simple_query = (
    Query(movie_index, weights={
        description_space: Param("description_weight"),
        title_space: Param("title_weight"),
        genre_space: Param("genre_weight"),
        recency_space: Param("recency_weight")
    })
    .find(movie)
    .similar(description_space.text, query_text_param)
    .similar(title_space.text, query_text_param)
    .similar(genre_space.text, query_text_param)
)

advanced_query = (
    Query(movie_index, weights={
        description_space: Param("description_weight"),
        title_space: Param("title_weight"),
        genre_space: Param("genre_weight"),
        recency_space: Param("recency_weight")
    })
    .find(movie)
    .similar(description_space.text, Param("description_query_text"))
    .similar(title_space.text, Param("title_query_text"))
    .similar(genre_space.text, Param("genre_query_text"))
)

In [9]:
df_parser = DataFrameParser(schema=movie, mapping={movie.release_timestamp: "timestamp"})

In [10]:
source: InMemorySource = InMemorySource(movie, parser=df_parser)
executor: InMemoryExecutor = InMemoryExecutor(sources=[source], indices=[movie_index])
app: InMemoryApp = executor.run()

In [11]:
source.put([movie_df])

## Run queries

### Helpers

In [12]:
def get_ordered_result_tuples(result: Result, top_n: int) -> list[tuple[int]]:
    return [(i+1, int(entity.id_.object_id)) for i, entity in enumerate(result.entities[:top_n])]

def get_movies_by_id_list(id_list_tuple: list[tuple[int]], df: pd.DataFrame) -> pd.DataFrame:
    if df.index.name != "id":
        df = df.set_index("id")
    result_df = df.loc[[id_tuple[1] for id_tuple in id_list_tuple]]
    result_df["order"] = [id_tuple[0] for id_tuple in id_list_tuple]
    return result_df

def parse_results(result: Result, df: pd.DataFrame, top_n: int = TOP_N) -> pd.DataFrame:
    id_tuples = get_ordered_result_tuples(result=result, top_n=top_n)
    return get_movies_by_id_list(id_list_tuple=id_tuples, df=df)

### Queries

With the simple query, I can search with my text in all of the fields

In [13]:
result: Result = app.query(
    simple_query,
    query_text="Heartfelt romantic comedy",
    description_weight=1,
    title_weight=1,
    genre_weight=1,
    recency_weight=0
)

In [14]:
parse_results(result, movie_df, 10)

,description,genres,title,release_year,timestamp,order
id,,,,,,
-2253176891792976545,When a group of people meets at the same party...,comedy romance,"Love, Surreal and Odd",2017,1.483229e+09,1
4482259520095813975,"An LA girl, unlucky in love, falls for an East...",romance comedy,Love Hard,2021,1.609459e+09,2
-5372923611380950453,Romantic anthology web series revolving around...,drama romance,Love Daily,2018,1.514765e+09,3
-4670188557169549123,"In this romantic comedy, several friends, each...",comedy romance,F*ck Love Too,2022,1.640995e+09,4
-6971815617200270214,Aspiring pop star Erica ends up as the enterta...,romance comedy,Resort to Love,2021,1.609459e+09,5
6458156743021149389,'Love Actually' follows the lives of eight ver...,drama comedy romance,Love Actually,2003,1.041379e+09,6
1439029578810365906,Exploration into the tense relationship of suc...,romance,Love or Money,2021,1.609459e+09,7
6001414845783902956,It tells the love story of two childhood sweet...,comedy romance,A Love So Beautiful,2017,1.483229e+09,8
-203728441379706695,Two young kids fall in love with each other. B...,romance drama,Endless Love,1981,3.471552e+08,9


After looking at the results, I see some titles I have already seen. I can bias towards recent titles by upweighting recency. Weights are normalised to have unit sum, so you don't have to worry about how you set them.

In [15]:
result: Result = app.query(
    simple_query,
    query_text="Heartfelt romantic comedy",
    description_weight=1,
    title_weight=1,
    genre_weight=1,
    recency_weight=3
)

In [16]:
parse_results(result, movie_df, 10)

,description,genres,title,release_year,timestamp,order
id,,,,,,
4482259520095813975,"An LA girl, unlucky in love, falls for an East...",romance comedy,Love Hard,2021,1.609459e+09,1
-6971815617200270214,Aspiring pop star Erica ends up as the enterta...,romance comedy,Resort to Love,2021,1.609459e+09,2
1439029578810365906,Exploration into the tense relationship of suc...,romance,Love or Money,2021,1.609459e+09,3
-4120066152600144910,"To save her small law firm, earnest lawyer Sus...",comedy romance,"Love, Guaranteed",2020,1.577837e+09,4
-1358543628890131127,While trying to make his sister's wedding day ...,comedy romance,Love Wedding Repeat,2020,1.577837e+09,5
-4945010711023394877,Love is as tough as it is sweet for a lovestru...,comedy drama,A Love So Beautiful,2020,1.577837e+09,6
762474985598598343,A rising black painter tries to break into a c...,romance drama,Really Love,2020,1.577837e+09,7
-1527455171103283110,Fashion assistant Maca has just about got her ...,comedy music romance drama,Sounds Like Love,2021,1.609459e+09,8
6268629262228820865,A 33-year-old fashion marketing director at a ...,drama romance,I Need Romance,2021,1.609459e+09,9


Still using the simple query, I can give more weight to spaces if I think my query is more related to that space - matches there should count more. Here I give additional weight to the genre, leave the description as is, and downweight the title as my query text is mostly a genre with some additional context. I keep recency with unit weight too, as I would like my results to be a bit biased towards recent movies.

In [17]:
result = app.query(
    simple_query,
    query_text="Heartfelt romantic comedy",
    description_weight=1,
    title_weight=0.1,
    genre_weight=2,
    recency_weight=1
)

In [18]:
parse_results(result, movie_df, 10)

,description,genres,title,release_year,timestamp,order
id,,,,,,
-4670188557169549123,"In this romantic comedy, several friends, each...",comedy romance,F*ck Love Too,2022,1.640995e+09,1
4482259520095813975,"An LA girl, unlucky in love, falls for an East...",romance comedy,Love Hard,2021,1.609459e+09,2
6869323869268244463,Short films follow young adults as they naviga...,drama romance comedy,Feels Like Ishq,2021,1.609459e+09,3
5997637689084803660,Incurable romantic Lotte's life is upended whe...,comedy romance,Just Say Yes,2021,1.609459e+09,4
-7468512594410561379,"In pursuit of both success and validation, a g...",romance comedy,Slay,2021,1.609459e+09,5
9007852042455380987,A blue-collar baker strikes up a relationship ...,comedy drama romance,The Baker and the Beauty,2020,1.577837e+09,6
-7276226089885610578,Lara Jean and Peter have just taken their roma...,romance comedy drama,To All the Boys: P.S. I Still Love You,2020,1.577837e+09,7
8761218778213996719,When professional ambitions clash with persona...,romance comedy drama,Love Aaj Kal,2020,1.577837e+09,8
-6971815617200270214,Aspiring pop star Erica ends up as the enterta...,romance comedy,Resort to Love,2021,1.609459e+09,9


With the advanced query, I can even supply different search terms for each attribute of the movie.

In [19]:
result = app.query(
    advanced_query,
    description_query_text="Heartfelt lovely romantic comedy for a cold autumn evening.",
    title_query_text="love",
    genre_query_text="drama comedy romantic",
    description_weight=1,
    title_weight=1,
    genre_weight=1,
    recency_weight=0
)

In [20]:
parse_results(result, movie_df, 10)

,description,genres,title,release_year,timestamp,order
id,,,,,,
6458156743021149389,'Love Actually' follows the lives of eight ver...,drama comedy romance,Love Actually,2003,1.041379e+09,1
-2128414429661167014,Rebellious Mickey and good-natured Gus navigat...,comedy drama romance,Love,2016,1.451606e+09,2
4482259520095813975,"An LA girl, unlucky in love, falls for an East...",romance comedy,Love Hard,2021,1.609459e+09,3
-203728441379706695,Two young kids fall in love with each other. B...,romance drama,Endless Love,1981,3.471552e+08,4
762474985598598343,A rising black painter tries to break into a c...,romance drama,Really Love,2020,1.577837e+09,5
-5372923611380950453,Romantic anthology web series revolving around...,drama romance,Love Daily,2018,1.514765e+09,6
6629114326765421907,Adam and Marklin’s 5-year relationship has gon...,comedy drama romance,Almost Love,2019,1.546301e+09,7
-4945010711023394877,Love is as tough as it is sweet for a lovestru...,comedy drama,A Love So Beautiful,2020,1.577837e+09,8
-1543140081073919556,"Near by Christmas, in an old and charming town...",romance european comedy,Love Is a Story,2015,1.420070e+09,9


And even give different weights to each subsearch if I really do care that the title is related to love but I am not really emotionally attached to my description of the movie I would want to see.

In [21]:
result = app.query(
    advanced_query,
    description_query_text="Heartfelt lovely romantic comedy for a cold autumn evening.",
    title_query_text="love",
    genre_query_text="drama comedy romantic",
    description_weight=0.2,
    title_weight=3,
    genre_weight=1,
    recency_weight=0
)

In [22]:
parse_results(result, movie_df, 10)

,description,genres,title,release_year,timestamp,order
id,,,,,,
-2128414429661167014,Rebellious Mickey and good-natured Gus navigat...,comedy drama romance,Love,2016,1.451606e+09,1
-7164587252458323203,The story of a family and the various situatio...,thriller drama,Love,2020,1.577837e+09,2
6458156743021149389,'Love Actually' follows the lives of eight ver...,drama comedy romance,Love Actually,2003,1.041379e+09,3
-203728441379706695,Two young kids fall in love with each other. B...,romance drama,Endless Love,1981,3.471552e+08,4
762474985598598343,A rising black painter tries to break into a c...,romance drama,Really Love,2020,1.577837e+09,5
6629114326765421907,Adam and Marklin’s 5-year relationship has gon...,comedy drama romance,Almost Love,2019,1.546301e+09,6
-4996069433895887278,"The story of Richard and Mildred Loving, an in...",drama romance,Loving,2016,1.451606e+09,7
4482259520095813975,"An LA girl, unlucky in love, falls for an East...",romance comedy,Love Hard,2021,1.609459e+09,8
7464380114309549848,In order to receive a bone marrow transplant q...,drama romance,Well-Intended Love,2019,1.546301e+09,9


Then I can bias again towards recent movies

In [23]:
result = app.query(
    advanced_query,
    description_query_text="Heartfelt lovely romantic comedy for a cold autumn evening.",
    title_query_text="love",
    genre_query_text="drama comedy romantic",
    description_weight=0.2,
    title_weight=3,
    genre_weight=1,
    recency_weight=5
)

In [24]:
parse_results(result, movie_df, 10)

,description,genres,title,release_year,timestamp,order
id,,,,,,
-7164587252458323203,The story of a family and the various situatio...,thriller drama,Love,2020,1.577837e+09,1
762474985598598343,A rising black painter tries to break into a c...,romance drama,Really Love,2020,1.577837e+09,2
4482259520095813975,"An LA girl, unlucky in love, falls for an East...",romance comedy,Love Hard,2021,1.609459e+09,3
-4945010711023394877,Love is as tough as it is sweet for a lovestru...,comedy drama,A Love So Beautiful,2020,1.577837e+09,4
-2128414429661167014,Rebellious Mickey and good-natured Gus navigat...,comedy drama romance,Love,2016,1.451606e+09,5
-4120066152600144910,"To save her small law firm, earnest lawyer Sus...",comedy romance,"Love, Guaranteed",2020,1.577837e+09,6
5891139576901071238,"In 1990s Turkey, a group of teenage outcasts b...",drama romance comedy,Love 101,2020,1.577837e+09,7
-1527455171103283110,Fashion assistant Maca has just about got her ...,comedy music romance drama,Sounds Like Love,2021,1.609459e+09,8
1303846551449642454,A-Cheng collects debts for a gangster and does...,romance drama,Man in Love,2021,1.609459e+09,9


Or maybe to older ones

In [25]:
result = app.query(
    advanced_query,
    description_query_text="Heartfelt lovely romantic comedy for a cold autumn evening.",
    title_query_text="love",
    genre_query_text="drama comedy romantic",
    description_weight=0.2,
    title_weight=3,
    genre_weight=1,
    recency_weight=-10
)

parse_results(result, movie_df, 10)

,description,genres,title,release_year,timestamp,order
id,,,,,,
-203728441379706695,Two young kids fall in love with each other. B...,romance drama,Endless Love,1981,347155200.0,1
927145628647552991,"Anil, a street singer, is humiliated and drive...",drama romance,Disco Dancer,1982,378691200.0,2
-1206271239451732125,An honest man dreams of a better life for his ...,romance crime drama,Ujala,1959,-347155200.0,3
-4146025651580076777,Two talented song-and-dance men team up after ...,romance comedy,White Christmas,1954,-504921600.0,4
6042318615022537162,"Brian Cohen is an average young Jewish man, bu...",comedy,Life of Brian,1979,283996800.0,5
1221097948770118142,Geeky student Arnie Cunningham falls for Chris...,horror thriller european,Christine,1983,410227200.0,6
2780969388900593033,Maharaj Brajbhan lives a wealthy lifestyle in ...,drama action romance,Bandie,1978,252460800.0,7
-5555982273817581438,Two small children and a ship's cook survive a...,romance action drama,The Blue Lagoon,1980,315532800.0,8
4870864013749376685,"In the 1930s, bored waitress Bonnie Parker fal...",crime drama action,Bonnie and Clyde,1967,-94694400.0,9
